<a href="https://colab.research.google.com/github/GeorgeM2000/Net2Net-NE/blob/master/Net2Net-NE/Net2Net-NE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Libraries & Tools***

In [1]:
import time
import networkx as nx
import numpy as np
import torch
import re
import math
import random
import gc
#import tensorflow as tf

from datetime import datetime

from torch import nn
from torch.nn import init
from torch.nn import functional as F

from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, confusion_matrix
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LogisticRegression

#from tensorflow.keras.preprocessing.text import Tokenizer
#from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
if torch.cuda.is_available():
    print("CUDA is available!")
else:
    print("CUDA is not available, using CPU.")

In [ ]:
gpu_id = 0
gpu = torch.device('cuda', gpu_id)
print(gpu)  # Should print: cuda:0

In [ ]:
print(torch.cuda.get_device_name(gpu))

In [ ]:
!pwd

# ***Global Variables & General Functionality***

In [5]:
dataset_name = "arxiv"
parent_path = f'Datasets/{dataset_name}/graph-v2'
parent_path = '/content'
graph_file = 'edges.txt'
categories_file = 'labels.txt'
vocab_file = 'voc.txt'

data_text_file  = "abstract.txt"
data_text_files = ["data-v3-500.txt", "data-v3-500C.txt", "YAKE10.txt", "YAKE5.txt", "RAKE10.txt", "RAKE5.txt", "RAKE10C.txt", "RAKE5C.txt", "TFIDF10.txt", "TFIDF5.txt", "PosR5.txt",
                   "PosR10.txt", "TextR5.txt", "TextR10.txt", "TopicR5.txt", "TopicR10.txt"]

log_file               = 'Net2Net-NE_Execution_Logs.txt'
link_pred_results_file = 'Net2Net-NE_Link_Pred_Res.txt'
node_clf_results_file  = 'Net2Net-NE_Node_Clf_Res.txt'


split_graph_file  = 'sgraph15.txt'
split_graph_files = ['sgraph15.txt', 'sgraph45.txt', 'sgraph75.txt']

test_graph_file   = 'tgraph85.txt'
test_graph_files  = ['tgraph85.txt', 'tgraph55.txt', 'tgraph25.txt']

In [6]:
word_num = 12619
MAX_LEN = 100 # Default value for single execution
MAX_LENS = [] # List to hold values for multiple executions

word_emb_dim = 200 # Original: 500
conv_dim = 200 # Orig: 500
kernel_num = 200
kernel_sizes = [1, 2, 3, 4, 5]
conv_drop = 0.2
enc_dim = 200 # Orig: 500
batch_size = 64
epoch_num = 50
l_rate = 1e-3
clf_ratio = [0.15, 0.45, 0.75]
clf_num = 5
train_classifier = True

In [ ]:
# Find the average number of words from each data text file
for txtf in ['abstract.txt']: # 1) ['data-v3-500.txt'] 2) data_text_files:
    total_word_count = 0
    total_lines = 0

    with open(f'{parent_path}/{txtf}', 'r', encoding='utf-8') as file:
        for line in file:
            total_word_count += len(re.findall(r"\b\w+\b", line))
            total_lines += 1

    mean_word_count = total_word_count / total_lines if total_lines > 0 else 0
    MAX_LENS.append(int(math.ceil(mean_word_count)))
    print(f'=== {txtf} ===')
    print("Mean word count:", math.ceil(mean_word_count))
    print(f'Total words: {total_word_count}')
    print()

In [ ]:
MAX_LENS

In [9]:
MAX_LEN = MAX_LENS[-1] # For single execution

In [ ]:
def get_vectors_from_file(file_path):
  vectors = {}

  with open(f'{file_path}', "r", encoding='utf-8') as f:
      for idx, line in enumerate(f):
          vector = list(map(float, line.strip().split()))  # Convert to list of floats
          vectors[idx] = vector  # Assign embedding to node idx

  return vectors

In [ ]:
zero_list = []
for i in range(0, word_emb_dim):
    zero_list.append(0)
zero_list = np.array(zero_list)

In [ ]:
# Create the edge list. Store the unique nodes in the list "nodes"
with open(f'{parent_path}/{graph_file}', 'r') as f:
  eedges = f.readlines()

edge_list = []
nodes = [] # "nodes" will contain all the unique nodes of the graph

for ee in eedges:
  edge_list.append(list(ee.split()))

for ll in edge_list:
  for ed in ll:
    if ed not in nodes:
      nodes.append(ed)
    else:
      continue

# ***Classify***

In [10]:
class TopKRanker(OneVsRestClassifier):
    def predict(self, X, top_k_list):
        probs = np.asarray(super(TopKRanker, self).predict_proba(X))
        all_labels = []
        for i, k in enumerate(top_k_list):
            probs_ = probs[i, :]
            labels = self.classes_[probs_.argsort()[-k:]].tolist()
            probs_[:] = 0
            probs_[labels] = 1
            all_labels.append(probs_)
        return np.asarray(all_labels)


class Classifier(object):
    def __init__(self, vectors, clf):
        self.embeddings = vectors
        self.clf = TopKRanker(clf)
        self.binarizer = MultiLabelBinarizer(sparse_output=True)

    def train(self, X, Y, Y_all):
        self.binarizer.fit(Y_all)
        X_train = [self.embeddings[x] for x in X]
        Y = self.binarizer.transform(Y)
        self.clf.fit(X_train, Y)

    def evaluate(self, X, Y):
        top_k_list = [len(l) for l in Y]
        Y_ = self.predict(X, top_k_list)
        Y = self.binarizer.transform(Y)
        # averages = ["micro", "macro", "samples", "weighted"]
        # f1_results = {}
        # pre_results = {}
        # rec_results = {}
        # acc_results = accuracy_score(Y, Y_)
        # f1_macro = f1_score(Y, Y_, average="macro")
        f1_micro = f1_score(Y, Y_, average="micro")
        # for average in averages:
        #      f1_results[average] = f1_score(Y, Y_, average=average)
        #     pre_results[average] = precision_score(Y, Y_, average=average)
        #     rec_results[average] = recall_score(Y, Y_, average=average)
        # print 'Results, using embeddings of dimensionality', len(self.embeddings[X[0]])
        # print '-------------------'
        # print('\nF1 Score: ')
        # print(f1_results)
        # print('\nPrecision Score:')
        # print(pre_results)
        # print('\nRecall Score:')
        # print(rec_results)
        # print('Accuracy Score:', acc_results)

        # return f1_results, pre_results, rec_results, acc_results
        return f1_micro
        # print '-------------------'

    ''' def evaluate(self, X, Y):
        top_k_list = [len(l) for l in Y] # For each label in Y, take its size (multi-label)
        Y_ = self.predict(X, top_k_list)
        Y = self.binarizer.transform(Y)
        averages = ["micro", "macro"]
        results = {}
        for average in averages:
            results[average] = f1_score(Y, Y_, average=average)
        return results '''

    def predict(self, X, top_k_list):
        X_ = np.asarray([self.embeddings[x] for x in X])
        Y = self.clf.predict(X_, top_k_list=top_k_list)
        return Y

    def split_train_evaluate(self, X, Y, train_precent, seed=0):
        state = np.random.get_state()

        training_size = int(train_precent * len(X))
        np.random.seed(seed)
        shuffle_indices = np.random.permutation(np.arange(len(X)))
        X_train = [X[shuffle_indices[i]] for i in range(training_size)]
        Y_train = [Y[shuffle_indices[i]] for i in range(training_size)]
        X_test = [X[shuffle_indices[i]] for i in range(training_size, len(X))]
        Y_test = [Y[shuffle_indices[i]] for i in range(training_size, len(X))]

        self.train(X_train, Y_train, Y)
        np.random.set_state(state)
        return self.evaluate(X_test, Y_test)


def load_embeddings(filename):
    fin = open(filename, 'r')
    node_num, size = [int(x) for x in fin.readline().strip().split()]
    vectors = {}
    while 1:
        l = fin.readline()
        if l == '':
            break
        vec = l.strip().split(' ')
        assert len(vec) == size + 1
        vectors[vec[0]] = [float(x) for x in vec[1:]]
    fin.close()
    assert len(vectors) == node_num
    return vectors


def read_node_label(filename):
    fin = open(filename, 'r')
    X = []
    Y = []
    while 1:
        l = fin.readline()
        if l == '':
            break
        vec = l.strip().split(' ')

        if len(vec) == 2:
            X.append(int(vec[0])) # X will contain the node IDs
            Y.append([int(v) for v in vec[1:]]) # For each node in the file Y will contain its corresponding labels (one or multiple labels)
    fin.close()
    return X, Y

# ***Utilities***

In [19]:
# Read node features from file
def read_node_fea(feature_path):
    fea = []
    fin = open(feature_path, 'r')
    for l in fin.readlines():
        vec = l.split()
        fea.append(np.array([float(x) for x in vec[1:]]))
    fin.close()
    return np.array(fea, dtype='float32')


def read_word_code(text_path, voca_path):

    # Read the vocabulary and store each word in that vocabulary in a list
    words = []
    fin = open(voca_path, 'r')
    for l in fin.readlines():
        words.append(l.strip())
    fin.close()

    print(f'Vocabulary size: {len(words)}')
    word_map = {words[i]: i for i in range(len(words))} # Each word will have a corresponding index
    pad_code = word_map['<eos>'] # The pad_code will be the index of the '<eos>' character

    # This segment reads a node's content and creates a token ID list for each node
    content_code = []
    fin = open(text_path, 'r')
    for l in fin.readlines():
        info = l.strip().split(' ') # Split the contents
        doc_code = [word_map[w] for w in info] # Get the ID (index) of each token/character

        # If there is variable length uncomment below
        """ if len(doc_code) > MAX_LEN:
            doc_code = doc_code[0: MAX_LEN]
        else:
            doc_code.extend([pad_code for _ in range(MAX_LEN - len(doc_code))]) """

        content_code.append(doc_code)

    print(f'Node contents: \n{len(content_code)}\n{len(content_code[0])}')
    return content_code, pad_code, len(words)
    # return np.array(content_code, dtype='int')


# load_text() method using Tokenizer(). This is my addition to the original code to replace the read_word_code() function
''' def load_text(self, text_path):
    text_file = open(text_path, 'rb').readlines()
    #text_data = [line.strip() for line in text_file]
    text_data = [line.decode('utf-8').strip() for line in text_file]

    tokenizer = Tokenizer(oov_token=None) # Default: Tokenizer()
    tokenizer.fit_on_texts(text_data)

    text = tokenizer.texts_to_sequences(text_data)
    text = pad_sequences(text, maxlen=MAX_LEN, padding="post", truncating='post') # Default: pad_sequences(text, maxlen=MAX_LEN, padding="post")

    num_vocab = len(tokenizer.word_index) + 1  # +1 for padding token
    num_nodes = len(text)
    return text, num_vocab, num_nodes '''

def load_text(self, text_path):
    """
    Adapting with adapt(text_data):

    vectorize_layer.adapt(text_data) analyzes text_data, builds a vocabulary, and assigns a unique integer ID to each word based on its frequency (most frequent words get lower IDs).
    Transforming with vectorize_layer(text_data):

    This maps each word in text_data to its corresponding integer token ID, producing a 2D array where each row represents a sequence of token IDs for a given input line, padded or truncated to MAX_LEN.
    """
    text_file = open(text_path, 'rb').readlines() # Read the file line by line
    for a in range(0, len(text_file)):
      text_file[a] = str(text_file[a]) # Convert its contents to string format

    vectorize_layer = tf.keras.layers.TextVectorization(
        max_tokens=None,  # Set a limit if needed
        output_mode='int',# The token IDs will be integers
        output_sequence_length=MAX_LEN
    )

    text_data = [line.strip() for line in text_file]
    text_data_size = len(text_data)

    if "data-v3-500" in self.text_filename: # If a file is too large to be processed at once, break it down into batches and apply the .adapt() method to each batch
      batch_size = int(text_data_size / 10)
      ranges = []
      start = 0
      while start < text_data_size:
          end = min(start + batch_size, text_data_size)  # Ensure the last range includes all remaining abstracts
          ranges.append([start, end])
          start = end

      ranges[-2][1] = ranges[-1][1]
      del ranges[-1]

      for range in ranges:
        vectorize_layer.adapt(text_data[range[0]:range[1]])

    else:
      vectorize_layer.adapt(text_data)

    text = vectorize_layer(text_data).numpy()
    num_vocab = len(vectorize_layer.get_vocabulary())
    #print(f'Vocabulary: {num_vocab}')
    num_nodes = len(text)

    return text, num_vocab, num_nodes


def fetch(content_code, ids, max_len, pad_code):
    # ids are the nodes
    # content_code is a list of lists representing the token IDs of a node's content

    code = []
    for id in ids: # For each node in nodes (ids)
        doc_code = content_code[id] # Take the list of token IDs of the current node

        # Apply padding or truncate the current node's token IDs to meet the limitation of MAX_LEN
        if len(doc_code) > max_len:
            doc_code = doc_code[0: max_len]
        else:
            doc_code.extend([pad_code for _ in range(max_len - len(doc_code))])

        code.append(doc_code)

    return code


def node_classification(hidden, idx, label, ratio):
    lr = Classifier(vectors=hidden, clf=LogisticRegression())
    res = lr.split_train_evaluate(idx, label, ratio)
    return res

# It combines multiple lists (that are inside another list, a 2D list) into a single list of unique elements, i.e., the union of all elements.
def exclusive_combine(*in_list):
    res = set()
    in_list = list(*in_list)
    for n_l in in_list:
        for i in n_l:
            res.add(i)
    return list(res)


def identity_map(n_list):
    id_dict = {}
    for i in range(len(n_list)):
        id_dict[n_list[i]] = i
    return id_dict


def agg_mean(M, id_dict, keys):
    idList = []
    for id in keys:
        idList.append(id_dict[id])

    return torch.mean(M[idList, :], 0, True)


def agg_max(M, id_dict, keys):
    idList = []
    for id in keys:
        idList.append(id_dict[id])
    res, _ = torch.max(M[idList, :], 0, True)
    return res

# ***Graph***

In [12]:
class MyGraph(object):
    def __init__(self, path, edgelist=True):
        self.neighbor_dict = {}

        # Create the graph
        if edgelist:
            fin = open(path, 'r')
            for l in fin.readlines():
                e = l.split()
                i, j = int(e[0]), int(e[1])

                # Undirected edges
                self.update_edge(i, j)
                self.update_edge(j, i)
            fin.close()

        # Convert each node's neighbors from dict to list
        for key in self.neighbor_dict.keys():
            self.neighbor_dict[key] = list(self.neighbor_dict[key])

        self.node_list = list(self.neighbor_dict.keys()) # node_list will contain all the node IDs sorted
        self.node_list.sort()
        self.node_num = len(self.node_list)

    def update_edge(self, i, j):
        if i in self.neighbor_dict:
            self.neighbor_dict[i].add(j)
        else:
            self.neighbor_dict[i] = {j}

        if j in self.neighbor_dict:
            self.neighbor_dict[j].add(i)
        else:
            self.neighbor_dict[j] = {i}

    def get_batches(self, batch_size):
        # np.random.seed(1)
        np.random.shuffle(self.node_list) # Shuffle the node_list
        num_batches = self.node_num // batch_size
        batch_list = []

        # Create "num_batches" number of batches
        for n in range(num_batches):
            batch_list.append(self.node_list[n * batch_size: (n + 1) * batch_size])

        # Create a final batch that contains the remaining nodes
        if self.node_num > num_batches * batch_size:
            batch_list.append(self.node_list[num_batches * batch_size:])

        self.node_list.sort() # Sort the node_list again after shuffling it
        return batch_list

    def get_neighbors(self, in_list): # in_list is a list of node IDs
        neighbors = [self.neighbor_dict[i] for i in in_list] # For each node get its neighbors. The result is a 2D list
        return exclusive_combine(neighbors)

    def diffuse(self, step, nodes):
        cur_list = nodes
        scale_list = [cur_list]
        for s in range(step):
            neighbors = self.get_neighbors(cur_list)
            cur_list = exclusive_combine([cur_list, neighbors])
            scale_list.append(cur_list)
        return scale_list  # From now to the past

    def statistic(self):
        neigh_num = []
        for n in self.node_list:
            neigh_num.append(len(self.neighbor_dict[n]))

        return np.max(neigh_num), np.min(neigh_num), np.mean(neigh_num)

# ***Models***

In [13]:
class MeanAggregator(nn.Module):

    def __init__(self, features, cur_device, gcn=False):

        super(MeanAggregator, self).__init__()
        self.features = features
        self.device = cur_device
        self.gcn = gcn

    def forward(self, nodes, to_neighs):
        samp_neighs = [samp_neigh + [nodes[i]] for i, samp_neigh in enumerate(to_neighs)]

        unique_nodes_list = exclusive_combine(samp_neighs)
        unique_nodes = {n: i for i, n in enumerate(unique_nodes_list)}
        # The mask for aggregation
        mask = torch.zeros(len(samp_neighs), len(unique_nodes), requires_grad=False, device=self.device)
        # The connections
        column_indices = [unique_nodes[n] for samp_neigh in samp_neighs for n in samp_neigh]
        row_indices = [i for i in range(len(samp_neighs)) for j in range(len(samp_neighs[i]))]
        mask[row_indices, column_indices] = 1
        # Normalize
        num_neigh = mask.sum(1, keepdim=True)
        mask = mask.div(num_neigh)

        embed_matrix = self.features(unique_nodes_list)
        to_feats = mask.mm(embed_matrix)
        return to_feats  # node_num * fea_dim

In [14]:
class EgoEncoder(nn.Module):
    def __init__(self, features, feature_dim, embed_dim, graph, aggregator, base_model=None):
        super(EgoEncoder, self).__init__()

        self.features = features
        self.feat_dim = feature_dim
        self.embed_dim = embed_dim
        self.graph = graph
        self.aggregator = aggregator
        if base_model is not None:
            self.base_model = base_model

        self.weight = nn.Parameter(torch.FloatTensor(self.feat_dim, embed_dim))
        init.xavier_uniform_(self.weight)

    def forward(self, nodes):
        to_neighs = [self.graph.neighbor_dict[node] for node in nodes]
        neigh_feats = self.aggregator.forward(nodes, to_neighs)
        combined = neigh_feats
        combined.mm(self.weight)
        combined = torch.tanh(combined)
        return combined  # node_num * emb_dim

In [15]:
class ContentCNN(nn.Module):
    def __init__(self, word_num, word_emb_dim, conv_dim, kernel_num, kernel_sizes, dropout, cur_device):
        super(ContentCNN, self).__init__()

        self.word_embeddings = nn.Embedding(word_num, word_emb_dim)
        self.word_embeddings.weight = nn.Parameter(torch.FloatTensor(word_num, word_emb_dim))
        self.word_embeddings.cuda(cur_device)

        # CNN with different kernel sizes
        self.conv_list = nn.ModuleList([nn.Conv2d(1, kernel_num, (K, word_emb_dim)) for K in kernel_sizes])

        self.dropout = nn.Dropout(dropout)
        # self.fc = nn.Linear(len(kernel_sizes) * kernel_num, conv_dim)
        self.weight = nn.Parameter(torch.FloatTensor(len(kernel_sizes) * kernel_num, conv_dim))
        self.device = cur_device

        init.xavier_uniform_(self.word_embeddings.weight)
        init.xavier_uniform_(self.weight)

    def conv_and_pool(self, x, conv):
        x_conv = conv(x)
        x_act = F.relu(x_conv).squeeze(3)  # (N, Co, W)
        x_pool = F.max_pool1d(x_act, x_act.size(2)).squeeze(2)
        return x_pool

    def forward(self, node_batch):
        query = torch.LongTensor(node_batch).cuda(self.device)
        x = self.word_embeddings(query)  # (N, W, D)

        x = x.unsqueeze(1)  # (N, Ci, W, D)

        x = [F.relu(conv(x)).squeeze(3) for conv in self.conv_list]  # [(N, Co, W), ...]*len(Ks)

        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]  # [(N, Co), ...]*len(Ks)

        x = torch.cat(x, 1)

        x = self.dropout(x)  # (N, len(Ks)*Co)
        # logit = self.fc(x)  # (N, C)
        logit = x.mm(self.weight)
        logit = torch.tanh(logit)
        return logit

# ***Net2Net-NE***

In [16]:
class Net2Net(nn.Module):
    def __init__(self, global_graph, features, encoder):
        super(Net2Net, self).__init__()
        self.graph = global_graph
        self.node_num = self.graph.node_num
        self.embed_dim = encoder.embed_dim
        self.features = features
        self.encoder = encoder
        self.xent = nn.CrossEntropyLoss()

        self.weight = nn.Parameter(torch.FloatTensor(self.embed_dim, self.node_num))
        init.xavier_uniform_(self.weight) # Original: init.xavier_uniform(self.weight)

    def forward(self, nodes):
        embeds = self.encoder(nodes)
        scores = embeds.mm(self.weight)
        return scores

    def loss(self, nodes, labels):
        scores = self.forward(nodes)
        return self.xent(scores, labels.squeeze())

    def evaluate(self, b_list, lab, ratio):
        self.eval() # It sets the model into evaluation mode, which: Disables dropout layers and makes batch normalization layers use their learned statistics instead of updating them

        hidden = [] # hidden represents the node embeddings
        idx = []
        for bat in b_list: # For each batch in the batch list
            h = self.encoder(bat)
            hidden.extend(h.detach().cpu().numpy())
            idx.extend(bat) # Adds the node indices from the batch into the idx list. This way, you keep track of which node embedding belongs to which node index

        print(f'Node embeddings info: \nLen:{len(hidden)}\n{len(hidden[0])}')
        print(f'Node indices info: \nLen:{len(idx)}\nMin:{min(idx)}\nMax:{max(idx)}')

        res = []
        for r in ratio: # For each splitting ratio, e.g., 0.15, 0.45, 0.75
            res.append(node_classification(hidden, # Take all the node embeddings
                                          np.arange(len(lab)),  # Create a list of all the node IDs. For link prediction, instead of len(lab), use the total number of nodes
                                          [lab[i] for i in idx], # For each node in idx take its corresponding label
                                          r))
        return res

    def evaluate_and_save_embeddings(self, b_list, graph, text_filename):
      self.eval()

      hidden = []
      idx = []
      for bat in b_list:
          h = self.encoder(bat)
          hidden.extend(h.detach().cpu().numpy())
          idx.extend(bat)

      # Save embeddings with a unique name
      embed_file = f"{parent_path}/Results/Net2Net-NE/embed_link_pred_{graph.split('.')[0]}_{text_filename.split('.')[0]}.txt"
      #embed_file = f"{parent_path}/Results/Net2Net-NE/embed_node_clf_{graph.split('.')[0]}_{text_filename.split('.')[0]}.txt"

      with open(embed_file, 'wb') as f:
        for i in range(len(idx)):
          if hidden[i]:
              f.write((' '.join(map(str, hidden)) + '\n').encode())
          else:
              f.write('\n'.encode()) # For link prediction
              f.write((' '.join(map(str, zero_list)) + '\n').encode()) # For node classification


# ***Train(Single Execution)***

In [17]:
# Read graph
graph = MyGraph(f'{parent_path}/{graph_file}')

# Read node labels
_, labels = read_node_label(f'{parent_path}/{categories_file}')

# Read node content (abstracts) and vocabulary of contents
node_content, pad_code, word_num = read_word_code(f'{parent_path}/{data_text_file}', f'{parent_path}/{vocab_file}')

features = ContentCNN(word_num, word_emb_dim, conv_dim, kernel_num, kernel_sizes, conv_drop, gpu)

agg1 = MeanAggregator(lambda nodes: features(fetch(node_content, nodes, MAX_LEN, pad_code)), gpu)
enc1 = EgoEncoder(lambda nodes: features(fetch(node_content, nodes, MAX_LEN, pad_code)), conv_dim, enc_dim, graph, agg1)

agg2 = MeanAggregator(lambda nodes: enc1(nodes), gpu)
enc2 = EgoEncoder(lambda nodes: enc1(nodes), enc1.embed_dim, enc_dim, graph, agg2, base_model=enc1)

c2n = Net2Net(graph, features, enc2)
c2n.cuda(gpu)

optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, c2n.parameters()), lr=l_rate)

batch_list = graph.get_batches(batch_size)

Vocabulary size: 11534
Node contents: 
1990
239


In [ ]:
start_time = datetime.now()
for e in range(epoch_num):
    avg_loss = []
    c2n.train()

    for batch in batch_list:
        optimizer.zero_grad()
        loss = c2n.loss(batch, torch.tensor(batch, dtype=torch.int64, device=gpu))
        loss.backward()
        optimizer.step()
        avg_loss.append(loss.item())

    print(f'Epoch: {e}')
    # Node classification results per epoch
    ''' res = c2n.evaluate(batch_list, labels, clf_ratio)
    total_time = np.around((time.time() - start) / 60.0)
    ls = np.mean(avg_loss)
    print('Epoch:', e, 'Loss:', ls, 'micro-F1:', np.around(res, 3), 'Time:', total_time, 'mins.') '''
    avg_loss.clear()

end_time = datetime.now()
print(f'Total Time: {((end_time - start_time).total_seconds()) / 60.0} min')
res = c2n.evaluate(batch_list, labels, clf_ratio)
print('micro-F1:', np.around(res, 3))
#c2n.evaluate_and_save_embeddings(batch_list)

## Link Prediction (For both single and multiple executions)


In [ ]:
embed_files = [
    [
        f"{parent_path}/Results/Net2Net-NE/embed_link_pred_sgraph{size}_{method}.txt"
        for method in ["data-v3-500", "data-v3-500C", "YAKE10", "PosR10", "PosR5", "YAKE5", "RAKE10", "RAKE5", "RAKE10C", "RAKE5C", "TFIDF5", "TFIDF10", "TextR10", "TextR5", "TopicR10", "TopicR5"]
    ]
    for size in ["15", "45", "75"]
]

# Initialize a log file to store the AUC results
with open(f'{parent_path}/Results/Net2Net-NE/{link_pred_results_file}', "a") as f:
    f.write("Embed File\tAUC Value\n")

for tgfi, tgf in enumerate(test_graph_files):
  for ef in embed_files[tgfi]:
      node2vec = {}

      # Load the embeddings from the current embed file
      with open(ef, 'rb') as f:
          for i, j in enumerate(f):
              if j.decode() != '\n':
                  node2vec[i] = list(map(float, j.strip().decode().split()))

      # Load the edges from the test graph file
      with open(f'{parent_path}/{tgf}', 'rb') as f:
          edges = [list(map(int, i.strip().decode().split())) for i in f]

      nodes = list(set([i for j in edges for i in j])) # All the unique nodes in "edges"

      # Calculate AUC
      a = 0
      b = 0
      for i, j in edges:
          if i in node2vec.keys() and j in node2vec.keys():
              dot1 = np.dot(node2vec[i], node2vec[j])
              random_node = random.sample(nodes, 1)[0]
              while random_node == j or random_node not in node2vec.keys():
                  random_node = random.sample(nodes, 1)[0]
              dot2 = np.dot(node2vec[i], node2vec[random_node])
              if dot1 > dot2:
                  a += 1
              elif dot1 == dot2:
                  a += 0.5
              b += 1

      auc_value = float(a) / b if b > 0 else 0
      print(f"AUC value for {ef.split('/')[-1]}: {auc_value}")

      # Log the result
      with open(f'{parent_path}/Results/Net2Net-NE/{link_pred_results_file}', "a") as f:
          f.write(f"{ef}\t{tgf}\t{auc_value}\n")

      gc.collect()

## Node Classification (For both single and multiple executions)


In [ ]:
embed_files = [
    f"{parent_path}/Results/Net2Net-NE/embed_node_clf_graph_{method}.txt"
    for method in ["data-v3-500", "data-v3-500C", "YAKE10", "PosR10", "PosR5", "YAKE5", "RAKE10", "RAKE5", "RAKE10C", "RAKE5C", "TFIDF5", "TFIDF10", "TextR10", "TextR5", "TopicR10", "TopicR5"]
]

with open(f'{parent_path}/{categories_file}', 'r') as f:
  tags = f.readlines() # "tags" will be a 2D list. Each sublist will have the form: nodeID     label

if train_classifier:

  clf_test_len = len(nodes) # The number of nodes will be the same in each run since we're using the whole graph and thus, all of its nodes

  for ef in embed_files:
    X = []
    Y = []
    new_vector = get_vectors_from_file(ef)

    for jk in range(0, clf_test_len):
      if str(jk) in nodes: # If the index "jk" is a node
        tag_list = tags[jk].strip().split() # For node "jk", take this info: jk     label
        # Y.append([(int)(i) for i in tags])
        lli = [str(i) for i in tag_list] # For node "jk", lli will contain all of its labels
        if len(lli) != 0:
          if np.array(new_vector[jk]).any() != np.array(zero_list).any(): # If there is no zero value in the embedding of "jk"
            X.append(jk)
            Y.append(lli[1:][0]) # Take the first label (if there are multiple) of node "jk"

    # This part of the code uses only the X and Y lists created above
    mi = {}
    ma = {}
    li1 = []
    li2 = []
    with open(f'{parent_path}/Results/DeepEmLAN/{node_clf_results_file}', 'a') as f:

      f.write(f"{ef.split('/')[-1]} \n")
      print(ef.split('/')[-1])

      for i in range(0, len(clf_ratio)): # Experiment with each ratio
        for j in range(0, clf_num): # clf_num = 5

          clf = Classifier(vectors=new_vector, # All node embeddings
                          clf=LogisticRegression())

          result = clf.split_train_evaluate(X, Y, clf_ratio[i])

          # Results
          li1.append(result['micro'])
          li2.append(result['macro'])

        mi[str(str(clf_ratio[i]) + '-micro')] = sum(li1) / clf_num
        ma[str(str(clf_ratio[i]) + '-macro')] = sum(li2) / clf_num

        print(mi)
        print(ma)
        print()

        f.writelines(str(str(mi)+str(ma)))
        f.write('\n')

        # Reinitialize the dictionaries and lists
        mi = {}
        ma = {}
        li1 = []
        li2 = []

    gc.collect()